# Atividade 3 - CIS

Essa atividade tem como objetivo implementar um Perceptron, esse é o tipo mais simples de rede neural. Para implementar essa rede foi necessário entender conceitos como feedforward e backpropagation. Para testar esse modelo foi utilizado um conjunto de dados de cartões de créditos. Ao final da atividade foi criado outro modelo usando um TensorFlow para fins de comparação.

In [3]:
import numpy as np 
import pandas as pd
from imblearn.over_sampling import SMOTE
from collections import Counter

# Explorando o Dado

A primeira etapa foi explorar o dataset para ver como os dados estavam distribuidos.

In [4]:
df = pd.read_csv('/kaggle/input/creditcard/creditcard.csv')

In [7]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [9]:
df["Class"].value_counts().tolist()

[284315, 492]

É possível perceber que o dado é extremamente desbalanceado, 284315 são transações legítimas e apenas 492 são transações fraudulentas. Além disso, as features do dataset não apresentam labels que nos ajudam a entender o problema.

----

# Criando o Perceptron

In [164]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

class Perceptron():
    def __init__(self, data, target):
        self.inputs = data
        self.outputs = np.zeros((1,len(target)))[0]
        self.target = target
        self.input_layer = self.inputs[0]
        self.v_hidden_layer = [0, 0]
        self.weights_l1 = np.random.rand(1,2*len(self.input_layer))[0]
        self.weights_l2 = [0.1,0.1]
        self.output_v = [0]
        
    def hidden_layer(self):
        for i in range(len(self.v_hidden_layer)):
            for j in range(len(self.input_layer)):
                self.v_hidden_layer[i] += self.input_layer[j] * self.weights_l1[i*len(self.input_layer)+j]
            self.v_hidden_layer[i] = sigmoid(self.v_hidden_layer[i])

    def output(self):
        self.output_v[0] = sigmoid(np.dot(self.v_hidden_layer, self.weights_l2))
        
    def forward(self):
        for k in range(len(self.inputs)):
            self.input_layer = self.inputs[k]
            self.hidden_layer()
            self.output()
            self.outputs[k] = self.output_v[0]
        
    def loss(self):
        return np.mean(1/2*(np.array(p.outputs) - np.array(p.target))**2)
        
    def backpropagation(self, lr):
        for i in range(len(self.target)):
            self.weights_l2[0] += (self.outputs[i]-self.target[i])*self.outputs[i]*(1-self.outputs[i])*self.v_hidden_layer[0] * lr
            self.weights_l2[1] += (self.outputs[i]-self.target[i])*self.outputs[i]*(1-self.outputs[i])*self.v_hidden_layer[1] * lr
            self.weights_l1[0] += (self.outputs[i]-self.target[i])*self.outputs[i]*(1-self.outputs[i])*self.weights_l2[0]*self.v_hidden_layer[0]*(1*self.v_hidden_layer[0])*self.input_layer[0] * lr
            self.weights_l1[1] += (self.outputs[i]-self.target[i])*self.outputs[i]*(1-self.outputs[i])*self.weights_l2[0]*self.v_hidden_layer[0]*(1*self.v_hidden_layer[0])*self.input_layer[1] * lr
            self.weights_l1[2] += (self.outputs[i]-self.target[i])*self.outputs[i]*(1-self.outputs[i])*self.weights_l2[0]*self.v_hidden_layer[1]*(1*self.v_hidden_layer[1])*self.input_layer[0] * lr
            self.weights_l1[3] += (self.outputs[i]-self.target[i])*self.outputs[i]*(1-self.outputs[i])*self.weights_l2[0]*self.v_hidden_layer[1]*(1*self.v_hidden_layer[1])*self.input_layer[1] * lr
            
    def train(self, epoch, lr):
        for i in range(epoch):
            self.forward()
            print(f'loss = {self.loss()}')
            self.backpropagation(lr)
        

In [115]:
# Separando as features do target
df_x = df.drop('Class', axis = 1)
df_y = df['Class']

In [117]:
data = df_x.values.tolist()

In [170]:
#Realizando o treinamento da rede
p = Perceptron(data, df_y)
p.train(4, 0.1)

loss = 0.1510726241683578


/tmp/ipykernel_32/868168670.py:2: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


loss = 0.49913625718468996
loss = 0.49913625718468996
loss = 0.49913625718468996


In [171]:
# Verificando como estava a saída do modelo
p.outputs

array([1., 1., 1., ..., 1., 1., 1.])

# Aplicando o Oversampling

Para balancear o conjunto de dados eu fiz uso da técnica de oversampling, já que na última atividade essa técnica trouxe uma melhora significativa para o modelo.

In [148]:
data_x = df_x.values
target = df_y.values

In [149]:
smote = SMOTE(random_state=42, k_neighbors=1)

X_smote, y_smote = smote.fit_resample(data_x, target)

In [166]:
#Realizando o treinamento da rede
p = Perceptron(X_smote, y_smote)
p.train(4, 0.1)

loss = 0.12624171141326673


/tmp/ipykernel_32/868168670.py:2: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


loss = 0.25
loss = 0.25
loss = 0.25


In [161]:
np.count_nonzero(p.outputs)

0

Em ambos os treinamentos os modelos não se saíram bem, acredito que esse problema seja causado por algum erro na implementação do algoritmo Perceptron. 

# Usando o TensorFlow

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

Nessa parte eu fiz uso do tensorflow para criar um modelo e tentar obter melhores resultados.

In [194]:
#Separando o dado em treinamento e validação
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

In [195]:
model = keras.Sequential([
    keras.layers.Dense(10, input_shape=(30,), activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']) #Como o dado ficou balanceado optei por continuar utilizando a acurácia como métrica
  
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
14216/14216 [==============================] - 24s 2ms/step - loss: 205.7812 - accuracy: 0.8770
Epoch 2/5
14216/14216 [==============================] - 23s 2ms/step - loss: 2.1502 - accuracy: 0.9372
Epoch 3/5
14216/14216 [==============================] - 24s 2ms/step - loss: 2.1841 - accuracy: 0.9441
Epoch 4/5
14216/14216 [==============================] - 24s 2ms/step - loss: 2.2221 - accuracy: 0.9475
Epoch 5/5
14216/14216 [==============================] - 23s 2ms/step - loss: 2.3101 - accuracy: 0.9489


In [196]:
model.evaluate(X_test, y_test)

3554/3554 [==============================] - 5s 1ms/step - loss: 1.8541 - accuracy: 0.9623


[1.8540579080581665, 0.9623041152954102]

Usando o TensorFlow com Oversampling foi possível consegui um resultado bem melhor (Acurácia de 96%).

# Conclusão 

Nessa atividade foi criado uma rede neural Perceptron do zero, para isso foi necessário entender os conceitos das redes neurais de forma mais profunda, como o backpropagation. Devido a isso, foi uma atividade que eu encontrei mais dificuldade.